In [11]:
from dotenv import load_dotenv
from typing import TypedDict

load_dotenv()


class GraphState(TypedDict):
    title: str
    filepath: str
    filetype: str
    page_numbers: list[int]
    page_elements: dict[int, dict[str, list[dict]]]
    page_metadata: dict[int, dict]
    page_summary: dict[int, str]
    images: list[str]
    images_summary: list[str]
    texts: list[str]
    texts_summary: list[str]

In [12]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
import re
import os
from langchain_experimental.text_splitter import SemanticChunker
from langchain_upstage.embeddings import UpstageEmbeddings
from langchain_upstage.chat_models import ChatUpstage
from langchain_core.documents import Document


def split_document(state: GraphState):
    file_path = state["filepath"]
    data = open(file_path, "r", encoding="utf-8")
    doc = data.read()

    title = os.path.basename(file_path)
    filetype = title.split(".")[-1]

    image_pattern = re.compile(r"!\[.*?\]\(.*?\)")

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
    ]

    if filetype == "md":  # -> List[Document]
        splitter = MarkdownHeaderTextSplitter(
            headers_to_split_on=headers_to_split_on, strip_headers=False
        )
    elif filetype == "txt":  # -> List[str]
        embeddings = UpstageEmbeddings(model="solar-embedding-1-large-passage")
        splitter = SemanticChunker(embeddings=embeddings)

    split_docs = splitter.split_text(doc)

    llm = ChatUpstage()
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large-passage")
    splited_docs = []
    for document in split_docs:
        if llm.get_num_tokens(document.page_content) > 3000:
            splitter2 = SemanticChunker(embeddings=embeddings)
            page_contents = splitter2.split_text(document.page_content)
            for content in page_contents:
                splited_docs.append(
                    Document(page_content=content, metadata=document.metadata)
                )
        else:
            splited_docs.append(document)
    num_pages = [x for x in range(len(splited_docs))]
    image_pattern = re.compile(r"!\[.*?\]\(.*?\)")

    page_elements = dict()
    page_metadata = dict()
    id = 0
    for i, split_doc in enumerate(splited_docs):
        page_elements[i] = []
        page_metadata[i] = split_doc.metadata
        images = image_pattern.findall(
            split_doc.page_content if filetype == "md" else split_doc
        )
        split_texts = image_pattern.split(
            split_doc.page_content if filetype == "md" else split_doc
        )

        for j, text in enumerate(split_texts):
            page_elements[i].append(
                {"id": id, "page": i, "type": "text", "content": text}
            )
            id += 1
            if j < len(images):
                page_elements[i].append(
                    {"id": id, "page": i, "type": "image", "content": images[j]}
                )
                id += 1

    return GraphState(
        page_numbers=num_pages,
        page_elements=page_elements,
        page_metadata=page_metadata,
        title=title,
        filetype=filetype,
    )

In [13]:
def extract_elements_per_page(state: GraphState):
    # GraphState 객체에서 페이지 요소들을 가져옵니다.
    page_elements = state["page_elements"]

    # 파싱된 페이지 요소들을 저장할 새로운 딕셔너리를 생성합니다.
    parsed_page_elements = dict()

    # 각 페이지와 해당 페이지의 요소들을 순회합니다.
    for key, page_element in page_elements.items():
        # 이미지, 테이블, 텍스트 요소들을 저장할 리스트를 초기화합니다.
        image_elements = []
        table_elements = []
        text_elements = []

        # 페이지의 각 요소를 순회하며 카테고리별로 분류합니다.
        for element in page_element:
            print(element)
            if element["type"] == "image":
                # 이미지 요소인 경우 image_elements 리스트에 추가합니다.
                image_url = element["content"].split("](")[1].split(")")[0]
                element["content"] = image_url
                image_elements.append(element)
            elif element["type"] == "table":
                # 테이블 요소인 경우 table_elements 리스트에 추가합니다.
                table_elements.append(element)
            else:
                # 그 외의 요소는 모두 텍스트 요소로 간주하여 text_elements 리스트에 추가합니다.
                text_elements.append(element)

        # 분류된 요소들을 페이지 키와 함께 새로운 딕셔너리에 저장합니다.
        parsed_page_elements[key] = {
            "image_elements": image_elements,
            "table_elements": table_elements,
            "text_elements": text_elements,
            "elements": page_element,  # 원본 페이지 요소도 함께 저장합니다.
        }

    # 파싱된 페이지 요소들을 포함한 새로운 GraphState 객체를 반환합니다.
    return GraphState(page_elements=parsed_page_elements)

In [14]:
def extract_page_text(state: GraphState):
    # 상태 객체에서 페이지 번호 목록을 가져옵니다.
    page_numbers = state["page_numbers"]

    # 추출된 텍스트를 저장할 딕셔너리를 초기화합니다.
    extracted_texts = dict()

    # 각 페이지 번호에 대해 반복합니다.
    for page_num in page_numbers:
        # 현재 페이지의 텍스트를 저장할 빈 문자열을 초기화합니다.
        extracted_texts[page_num] = ""

        # 현재 페이지의 모든 텍스트 요소에 대해 반복합니다.
        for element in state["page_elements"][page_num]["text_elements"]:
            # 각 텍스트 요소의 내용을 현재 페이지의 텍스트에 추가합니다.
            extracted_texts[page_num] += element["content"]

    # 추출된 텍스트를 포함한 새로운 GraphState 객체를 반환합니다.
    return GraphState(texts=extracted_texts)

In [15]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from summarizer import Summarizer
from utils import load_yaml


def create_text_summary(state: GraphState):
    # state에서 텍스트 데이터를 가져옵니다.
    texts = state["texts"]
    config = load_yaml("../config/GraphState.yaml")
    # 요약된 텍스트를 저장할 딕셔너리를 초기화합니다.
    text_summary = dict()
    # texts.items()를 페이지 번호(키)를 기준으로 오름차순 정렬합니다.
    sorted_texts = sorted(texts.items(), key=lambda x: x[0])

    # 각 페이지의 텍스트를 Document 객체로 변환하여 입력 리스트를 생성합니다.
    inputs = [Document(page_content=text) for _, text in sorted_texts]

    summarizer = Summarizer(config=config, summary_type="only_cod", use_cod=True)

    summaries = summarizer.chain_of_density(inputs)

    # 생성된 요약을 페이지 번호와 함께 딕셔너리에 저장합니다.
    for page_num, summary in enumerate(summaries):
        text_summary[page_num] = summary

    # 요약된 텍스트를 포함한 새로운 GraphState 객체를 반환합니다.
    return GraphState(text_summary=text_summary)

In [16]:
def create_image_summary_data_batches(state: GraphState):
    # 이미지 요약을 위한 데이터 배치를 생성하는 함수
    data_batches = []
    images = {}

    # 페이지 번호를 오름차순으로 정렬
    page_numbers = sorted(list(state["page_elements"].keys()))

    for page_num in page_numbers:
        # 각 페이지의 요약된 텍스트를 가져옴
        text = state["text_summary"][page_num]
        # 해당 페이지의 모든 이미지 요소에 대해 반복
        for image_element in state["page_elements"][page_num]["image_elements"]:
            # 이미지 ID를 정수로 변환
            image_id = int(image_element["id"])

            # 데이터 배치에 이미지 정보, 관련 텍스트, 페이지 번호, ID를 추가
            data_batches.append(
                {
                    "image": image_element["content"],  # 이미지 파일 경로
                    "text": text,  # 관련 텍스트 요약
                    "page": page_num,  # 페이지 번호
                    "id": image_id,  # 이미지 ID
                }
            )

            # images 딕셔너리에 이미지 요소 추가
            images[image_id] = image_element["content"]

    # 생성된 데이터 배치와 이미지를 GraphState 객체에 담아 반환
    return GraphState(image_summary_data_batches=data_batches, images=images)

In [17]:
from langchain_teddynote.models import MultiModal
from langchain_core.runnables import chain


@chain
def extract_image_summary(data_batches):
    # 객체 생성
    llm = ChatOpenAI(
        temperature=0,  # 창의성 (0.0 ~ 2.0)
        model_name="gpt-4o-mini",  # 모델명
    )

    system_prompt = """You are an expert in extracting useful information from IMAGE.
    With a given image, your task is to extract key entities, summarize them, and write useful information that can be used later for retrieval in Korean."""

    image_paths = []
    system_prompts = []
    user_prompts = []

    for data_batch in data_batches:
        context = data_batch["text"]
        image_path = data_batch["image"]
        user_prompt_template = f"""Here is the context related to the image: {context}
        
###

Output Format:

<image>
<title>
<summary>
<entities> 
</image>

"""
        image_paths.append(image_path)
        system_prompts.append(system_prompt)
        user_prompts.append(user_prompt_template)

    multimodal_llm = MultiModal(llm)

    """answer = multimodal_llm.batch(
        image_paths, system_prompts, user_prompts, display_image=False
    )"""

    answers = []

    # 각 배치를 개별적으로 처리
    for i in range(len(image_paths)):
        try:
            # 단일 이미지에 대한 질의 실행
            result = multimodal_llm.batch(
                [image_paths[i]],
                [system_prompts[i]],
                [user_prompts[i]],
                display_image=False,
            )
            answers.extend(result)
        except Exception as e:
            print(f"배치 {i} 처리 중 오류 발생: {str(e)}")
            # 오류 발생 시 빈 문자열 추가
            answers.append("")

    return answers

In [18]:
import logging


def create_image_summary(state: GraphState):
    image_summary_output = dict()

    try:
        image_summaries = extract_image_summary.invoke(
            state["image_summary_data_batches"],
        )

        for data_batch, image_summary in zip(
            state["image_summary_data_batches"], image_summaries
        ):
            image_summary_output[data_batch["id"]] = image_summary
    except Exception as e:
        logging.error(f"이미지 요약 생성 중 오류 발생: {str(e)}")

    return GraphState(image_summary=image_summary_output)

In [19]:
from langchain_openai import ChatOpenAI
import os
from langchain_community.document_transformers.openai_functions import (
    create_metadata_tagger,
)
from langchain.docstore.document import Document
from datetime import datetime
from utils import load_yaml
from metadata_properties import Property1

prompt = load_yaml("../prompts/metadatatagger.yaml")["prompt"]
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.1)


def create_metadata(state: GraphState):
    page_num = state["page_numbers"]

    split_docs = []
    for num in page_num:
        texts = state["texts"][num]
        metadata = state["page_metadata"][num]

        data = "\n".join(texts)

        if "images_summary" in state and num in state["images_summary"]:
            image_summary = state["images_summary"][num]
            data += f"\n{image_summary}"

        split_docs.append(Document(page_content=data, metadata=metadata))

    # ------------------------------------------------------------------
    merged_documents = []
    for doc in split_docs:

        merged_content = (
            f"metadata:{doc.metadata}\n\n ---- \n page_content:{doc.page_content}"
        )

        merged_documents.append(
            Document(page_content=merged_content, metadata=doc.metadata)
        )

    # ------------------------------------------------------------------
    document_transformer = create_metadata_tagger(Property1, llm)
    enhanced_documents = document_transformer.transform_documents(
        merged_documents, prompt=prompt
    )

    metadata = dict()
    for i in range(len(split_docs)):
        metadata[i] = []
        metadata[i] = enhanced_documents[i].metadata

    return GraphState(page_metadata=metadata)

In [ ]:
import json
import os
import logging
from typing import List

# 로깅 설정
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)


def save_state_to_json(state, file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(state, f, ensure_ascii=False, indent=4)


def get_files_by_type(directory: str, file_type: str) -> List[str]:
    return [
        os.path.join(directory, f)
        for f in os.listdir(directory)
        if f.endswith(f".{file_type}")
    ]


def process_file(file_path: str, output_path: str):
    try:
        state = GraphState(filepath=file_path)

        logging.info("split_document 시작")
        state_out = split_document(state)
        state.update(state_out)

        logging.info("extract_elements_per_page 시작")
        state_out = extract_elements_per_page(state)
        state.update(state_out)

        logging.info("extract_page_text 시작")
        state_out = extract_page_text(state)
        state.update(state_out)

        logging.info("create_text_summary 시작")
        state_out = create_text_summary(state)
        state.update(state_out)

        has_image_elements = any(
            "image_elements" in page and page["image_elements"]
            for page in state["page_elements"].values()
        )

        if has_image_elements:
            logging.info("create_image_summary_data_batches 시작")
            state_out = create_image_summary_data_batches(state)
            state.update(state_out)

            logging.info("create_image_summary 시작")
            state_out = create_image_summary(state)
            state.update(state_out)
        else:
            logging.info("이미지 요소가 없어 이미지 요약 과정을 건너뜁니다.")

        logging.info("create_metadata 시작")
        state_out = create_metadata(state)
        state.update(state_out)

        filename = os.path.splitext(os.path.basename(file_path))[0]
        json_path = f"{output_path}/{filename}.json"
        save_state_to_json(state, json_path)

        logging.info(f"파일 처리 완료: {file_path}")
    except Exception as e:
        logging.error(f"파일 처리 중 오류 발생: {file_path}. 오류: {str(e)}")
        logging.exception("상세 오류 정보:")
        raise Exception(f"파일 처리 중 오류로 인해 프로그램을 중단합니다: {file_path}")


def main():
    config = load_yaml("../config/GraphState.yaml")
    base_path = config["settings"]["base_path"]
    edit_path = config["settings"]["edit_path"]
    category_id = config["settings"]["category_id"]
    filetype = config["settings"]["filetype"]

    output_path = os.path.join(edit_path, category_id, "json")
    os.makedirs(output_path, exist_ok=True)
    input_directory = os.path.join(base_path, category_id, filetype)

    files = get_files_by_type(input_directory, filetype)

    for file in files:
        filename = os.path.splitext(os.path.basename(file))[0]
        json_path = f"{output_path}/{filename}.json"

        if os.path.exists(json_path):
            logging.info(f"이미 처리된 파일입니다. 건너뜁니다: {file}")
            continue

        process_file(file, output_path)


main()